In [1]:
# ────────────────────────────────────────────────────────────────
# 02_change_point_model.ipynb
# Simple SINGLE change point model with PyMC
# ────────────────────────────────────────────────────────────────

import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt

RANDOM_SEED = 20232023
rng = np.random.default_rng(RANDOM_SEED)

# ─── Load prepared data ─────────────────────────────────────────────
df = pd.read_csv("../data/processed/brent_model_ready.csv", parse_dates=["Date"])
df = df.dropna(subset=["price"])

y = df["price"].values          # modeling prices directly (you can switch to log_price)
N = len(y)
t = np.arange(N)

print(f"Modeling {N} days of data")

# ─── Model ──────────────────────────────────────────────────────────
with pm.Model() as model_single:

    # Change point (day index)
    tau = pm.DiscreteUniform("tau", lower=10, upper=N-10)

    # Means before & after
    mu_1 = pm.Normal("mu_1", mu=70, sigma=30)
    mu_2 = pm.Normal("mu_2", mu=70, sigma=30)

    # Common noise (you can make two sigmas later)
    sigma = pm.HalfNormal("sigma", sigma=15)

    # Switch mean
    mu = pm.math.switch(t < tau, mu_1, mu_2)

    # Likelihood
    obs = pm.Normal("obs", mu=mu, sigma=sigma, observed=y)

    # Sample!
    trace = pm.sample(
        draws=2000,
        tune=2000,
        chains=4,
        target_accept=0.92,
        random_seed=RANDOM_SEED,
        return_inferencedata=True
    )

# ─── Diagnostics ────────────────────────────────────────────────────
print(az.summary(trace, var_names=["tau", "mu_1", "mu_2", "sigma"]))

az.plot_trace(trace, var_names=["tau", "mu_1", "mu_2", "sigma"])
plt.tight_layout()
plt.show()

az.plot_posterior(trace, var_names=["tau", "mu_1", "mu_2"])
plt.show()

# ─── Plot posterior change point probability ────────────────────────
tau_samples = trace.posterior["tau"].values.flatten()

plt.figure(figsize=(12,5))
plt.hist(tau_samples, bins=np.arange(0, N+1, 5), density=True, color="teal", alpha=0.7)
plt.title("Posterior probability of change point location")
plt.xlabel("Day index (0 = start of dataset)")
plt.ylabel("Posterior density")

# Show real dates on top
xticks_idx = np.linspace(0, N, 8, dtype=int)
xtick_dates = df["Date"].iloc[xticks_idx].dt.strftime("%Y-%m")
plt.xticks(xticks_idx, xtick_dates, rotation=30)

plt.tight_layout()
plt.show()

# Most probable change point
map_tau = int(az.summary(trace)["mean"]["tau"])
print(f"MAP change point day index: {map_tau}")
print("Date:", df["Date"].iloc[map_tau])

c:\Users\Hp\Downloads\Tenx_academy\brent-oil-change-point-analysis\.venv\Lib\site-packages\arviz\__init__.py:50: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/brent_model_ready.csv'